# 如何使用PaddleHub进行图像分类
# 简介

图像分类是计算机视觉的重要领域，它的目标是将图像分类到预定义的标签。近期，许多研究者提出很多不同种类的神经网络，并且极大的提升了分类算法的性能。本文以Kaggle的猫狗分类数据集为例子，介绍如何使用PaddleHub进行图像分类。



In [3]:
'''
通过关键字在服务端检索匹配的模型
'''
!hub search resnet

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2020-04-04 18:01:30,310-INFO: font search path ['/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/afm', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/pdfcorefonts']
2020-04-04 18:01:30,646-INFO: generated new fontManager

+--------------------------------+----------+----------+---------------------------+
|          ResourceName          |   Type   | Version  |          Summary          |
+--------------------------------+----------+----------+---------------------------+
|     resnet_v2_50_imagenet      |  Module  |  1.0.1   |ResN

# Step1、加载预训练模型

由于猫狗分类是一个图像分类任务，因此我们使用经典的ResNet-50作为预训练模型。PaddleHub提供了丰富的图像分类预训练模型，包括了最新的神经网络架构搜索类的PNASNet，我们推荐您尝试不同的预训练模型来获得更好的性能。

PaddleHub 还有着许多的图像分类预训练模型，更多信息参见PaddleHub官方网站

加载module，并获取module的上下文环境，包括输入和输出的变量，以及Paddle Program；

In [4]:

import paddlehub as hub

module = hub.Module(name="resnet_v2_50_imagenet")

input_dict, output_dict, program = module.context(
    trainable=True)

[2020-04-04 18:01:32,986] [    INFO] - Installing resnet_v2_50_imagenet module


[==================================================] 100.00%
Uncompress /home/aistudio/.paddlehub/tmp/tmppobhgwyu/resnet_v2_50_imagenet
[==================================================] 100.00%


[2020-04-04 18:01:44,252] [    INFO] - Successfully installed resnet_v2_50_imagenet-1.0.1
[2020-04-04 18:01:44,580] [    INFO] - 267 pretrained paramaters loaded by PaddleHub


# Step2、数据准备
接着需要加载图片数据集。为了快速体验，我们直接加载paddlehub提供的猫狗分类数据集，如果想要使用自定义的数据进行体验，请查看[适配自定义数据](http://github.com/PaddlePaddle/PaddleHub/wiki/PaddleHub%E9%80%82%E9%85%8D%E8%87%AA%E5%AE%9A%E4%B9%89%E6%95%B0%E6%8D%AE%E5%AE%8C%E6%88%90F)

如果想加载自定义数据集完成迁移学习，详细参见[自定义数据集](https://github.com/PaddlePaddle/PaddleHub/wiki/PaddleHub%E9%80%82%E9%85%8D%E8%87%AA%E5%AE%9A%E4%B9%89%E6%95%B0%E6%8D%AE%E5%AE%8C%E6%88%90FineTune)

接着生成一个图像分类的reader，reader负责将dataset的数据进行预处理，接着以特定格式组织并输入给模型进行训练。

当我们生成一个图像分类的reader时，需要指定输入图片的大小


In [5]:

# 直接用PaddleHub提供的数据集
dataset = hub.dataset.DogCat()

data_reader = hub.reader.ImageClassificationReader(
    image_width=module.get_expected_image_width(),
    image_height=module.get_expected_image_height(),
    images_mean=module.get_pretrained_images_mean(),
    images_std=module.get_pretrained_images_std(),
    dataset=dataset)

[==================================================] 100.00%
Uncompress /home/aistudio/.paddlehub/dataset/dog-cat.tar.gz
[==================================================] 100.00%


[2020-04-04 18:02:34,783] [    INFO] - Dataset label map = {'cat': 0, 'dog': 1}


# Step3、配置策略


在进行Finetune前，我们可以设置一些运行时的配置，例如如下代码中的配置，表示：

* `use_cuda`：设置为False表示使用CPU进行训练。如果您本机支持GPU，且安装的是GPU版本的PaddlePaddle，我们建议您将这个选项设置为True；

* `epoch`：要求Finetune的任务只遍历1次训练集；

* `batch_size`：每次训练的时候，给模型输入的每批数据大小为32，模型训练时能够并行处理批数据，因此batch_size越大，训练的效率越高，但是同时带来了内存的负荷，过大的batch_size可能导致内存不足而无法训练，因此选择一个合适的batch_size是很重要的一步；

* `log_interval`：每隔10 step打印一次训练日志；

* `eval_interval`：每隔50 step在验证集上进行一次性能评估；

* `checkpoint_dir`：将训练的参数和数据保存到cv_finetune_turtorial_demo目录中；

* `strategy`：使用DefaultFinetuneStrategy策略进行finetune；

更多运行配置，请查看[RunConfig](https://github.com/PaddlePaddle/PaddleHub/wiki/PaddleHub-API:-RunConfig)

同时PaddleHub提供了许多优化策略，如`AdamWeightDecayStrategy`、`ULMFiTStrategy`、`DefaultFinetuneStrategy`等，详细信息参见[策略](https://github.com/PaddlePaddle/PaddleHub/wiki/PaddleHub-API:-Strategy)


In [6]:

config = hub.RunConfig(
    use_cuda=False,                              #是否使用GPU训练，默认为False；
    num_epoch=1,                                #Fine-tune的轮数；
    checkpoint_dir="cv_finetune_turtorial_demo",#模型checkpoint保存路径, 若用户没有指定，程序会自动生成；
    batch_size=32,                              #训练的批大小，如果使用GPU，请根据实际情况调整batch_size；
    eval_interval=50,                           #模型评估的间隔，默认每100个step评估一次验证集；
    strategy=hub.finetune.strategy.DefaultFinetuneStrategy())  #Fine-tune优化策略；

[2020-04-04 18:02:34,789] [    INFO] - Checkpoint dir: cv_finetune_turtorial_demo


# Step4、迁移组网

有了合适的预训练模型和准备要迁移的数据集后，我们开始组建一个Task。

由于猫狗分类是一个二分类的任务，而我们下载的分类module是在ImageNet数据集上训练的千分类模型，所以我们需要对模型进行简单的微调，把模型改造为一个二分类模型：


1. 从输出变量中找到特征图提取层feature_map；
2. 在feature_map后面接入一个全连接层，生成Task；

In [7]:



feature_map = output_dict["feature_map"]

feed_list = [input_dict["image"].name]

#ImageClassifierTask：通用的分类任务Task，该Task基于输入的特征，添加一个或多个全连接层来创建一个分类任务用于Fine-tune
task = hub.ImageClassifierTask(
    data_reader=data_reader, #提供数据的reader
    feed_list=feed_list,     #待feed变量的名字列表
    feature=feature_map,     #输入的特征矩阵
    num_classes=dataset.num_labels, #分类任务的类别数量
    config=config)                  #运行配置


2020-04-04 18:02:35,138-WARNING: paddle.fluid.layers.py_reader() may be deprecated in the near future. Please use paddle.fluid.io.DataLoader.from_generator() instead.
[2020-04-04 18:02:35,290] [    INFO] - Strategy with scheduler: {'warmup': 0.0, 'linear_decay': {'start_point': 1.0, 'end_learning_rate': 0.0}, 'noam_decay': False, 'discriminative': {'blocks': 0, 'factor': 2.6}, 'gradual_unfreeze': 0, 'slanted_triangle': {'cut_fraction': 0.0, 'ratio': 32}}, regularization: {'L2': 0.001, 'L2SP': 0.0, 'weight_decay': 0.0} and clip: {'GlobalNorm': 0.0, 'Norm': 0.0}
[2020-04-04 18:02:39,780] [    INFO] - Try loading checkpoint from cv_finetune_turtorial_demo/ckpt.meta
[2020-04-04 18:02:40,889] [    INFO] - PaddleHub model checkpoint loaded. current_epoch=2, global_step=564, best_score=0.99105
[2020-04-04 18:02:40,890] [    INFO] - PaddleHub finetune start
[2020-04-04 18:02:40,891] [    INFO] - PaddleHub finetune finished.


# Step5、Fine-tune

In [8]:
run_states = task.finetune_and_eval()

[2020-04-04 18:02:40,895] [    INFO] - PaddleHub finetune start
[2020-04-04 18:02:40,896] [    INFO] - PaddleHub finetune finished.


# Step6、预测

当Finetune完成后，我们使用模型来进行预测，先通过以下命令来获取测试的图片

注意：以下示例仍然以猫狗分类为例子，其他数据集所用的测试图片请自行准备

In [9]:
!wget --no-check-certificate https://paddlehub.bj.bcebos.com/resources/test_img_cat.jpg
!wget --no-check-certificate https://paddlehub.bj.bcebos.com/resources/test_img_dog.jpg

--2020-04-04 18:02:41--  https://paddlehub.bj.bcebos.com/resources/test_img_cat.jpg
Resolving paddlehub.bj.bcebos.com (paddlehub.bj.bcebos.com)... 182.61.200.195, 182.61.200.229
Connecting to paddlehub.bj.bcebos.com (paddlehub.bj.bcebos.com)|182.61.200.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36914 (36K) [image/jpeg]
Saving to: ‘test_img_cat.jpg.1’

test_img_cat.jpg.1  100%[===================>]  36.05K  --.-KB/s    in 0.01s   

2020-04-04 18:02:41 (3.56 MB/s) - ‘test_img_cat.jpg.1’ saved [36914/36914]

--2020-04-04 18:02:41--  https://paddlehub.bj.bcebos.com/resources/test_img_dog.jpg
Resolving paddlehub.bj.bcebos.com (paddlehub.bj.bcebos.com)... 182.61.200.195, 182.61.200.229
Connecting to paddlehub.bj.bcebos.com (paddlehub.bj.bcebos.com)|182.61.200.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113952 (111K) [image/jpeg]
Saving to: ‘test_img_dog.jpg.1’

test_img_dog.jpg.1  100%[===================>] 111.28K  --.-KB/

In [10]:
import numpy as np

data = ["test_img_dog.jpg"]
label_map = dataset.label_dict()

index = 0
run_states = task.predict(data=data)
results = [run_state.run_results for run_state in run_states]

for batch_result in results:
    print(batch_result)
    batch_result = np.argmax(batch_result, axis=2)[0]
    print(batch_result)
    for result in batch_result:
        index += 1
        result = label_map[result]
        print("input %i is %s, and the predict result is %s" %
              (index, data[index - 1], result))

[2020-04-04 18:02:41,713] [    INFO] - Load the best model from cv_finetune_turtorial_demo/best_model
2020-04-04 18:02:42,041-WARNING: paddle.fluid.layers.py_reader() may be deprecated in the near future. Please use paddle.fluid.io.DataLoader.from_generator() instead.
[2020-04-04 18:02:43,097] [    INFO] - PaddleHub predict start
[2020-04-04 18:02:43,528] [    INFO] - PaddleHub predict finished.


[array([[0.00153231, 0.9984676 ]], dtype=float32)]
[1]
input 1 is test_img_dog.jpg, and the predict result is dog
